In [13]:
import json
import glob
from collections import Counter

In [22]:
files = glob.glob(f"../data/data_saha/*/*/*.json")
print (len(files))

2460


In [23]:
speakers = {}
for filename in files:
    with open (filename, "r") as f:
        data = json.load(f)
    tmp = data["header"]["speakers"]
    for t in tmp:
        if t in speakers.keys():
            if filename not in speakers[t]:
                speakers[t].append(filename)
        else:
            speakers[t] = [filename]
# speakers = list(set(speakers))
# speakers.sort()

In [24]:
len(speakers)

5064

In [25]:
with open ("../data/all_speakers.json", "w") as f:
    json.dump(speakers, f, indent=4)

In [6]:
connected = {}
lastnames = []
false_pos = []
for s in speakers:
    if len(s.split()) > 1 and len(s.split()) < 5:
        item = s
        if " AT " not in item and " BY " not in item and "REPLY" not in item and "ADDRESSES" not in item and "ARGUMENT" not in item and "REEXAMINATION" not in item:
            last = s.split()[-1]
            if len(last) > 2:
                lastnames.append(last)
    if len(s.split()) > 4:
        false_pos.append(s)
potential = Counter(lastnames)
connected = {}
connected_more = {}
for p in potential:
    name = potential[p]
    if name == 2:
        for s in speakers:
            while "  " in s:
                s = s.replace("  ", " ")
            s = s.replace("NO FURTHER QUESTIONS BY ", "").replace("NO FURTHER CROSSEXAMINATION BY ", "").replace("EXAMINAION BY ", "").replace("CROSS EXAMINATION ", "").replace("NO QUESTIONS BY ", "").replace("QUESTIONS BY ", "")
            s = s.replace("NO CROSSEXAMINATION BY ", "").replace("CROSSEXAMINATION BY ", "").replace("FURTHER CROSSEXAMINATION ", "").replace("CROSSEXAMINATION ", "").replace("NO EXAMINATION BY ", "")
            s = s.replace("NO REEXAMINATION BY ", "").replace("REEXAMINATION BY ", "")
            if len(s.split()) > 0:
                if p == s.split()[-1]:
                    # item = s
                    # if " BY " not in item and "REPLY" not in item and "ADDRESSES" not in item and "ARGUMENT" not in item and "REEXAMINATION" not in item
                    if p in connected:
                        if s not in connected[p]:
                            connected[p].append(s)
                    else:
                        connected[p] = [s]
    elif name > 2:
        for s in speakers:
            while "  " in s:
                s = s.replace("  ", " ")
            s = s.replace("NO FURTHER QUESTIONS BY ", "").replace("NO FURTHER CROSSEXAMINATION BY ", "").replace("EXAMINAION BY ", "").replace("CROSS EXAMINATION ", "").replace("NO QUESTIONS BY ", "").replace("QUESTIONS BY ", "")
            s = s.replace("NO CROSSEXAMINATION BY ", "").replace("CROSSEXAMINATION BY ", "").replace("FURTHER CROSSEXAMINATION ", "").replace("CROSSEXAMINATION ", "").replace("NO EXAMINATION BY ", "")
            s = s.replace("NO REEXAMINATION BY ", "").replace("REEXAMINATION BY ", "")
            if len(s.split()) > 0:
                if p == s.split()[-1]:
                
                    if p in connected_more:
                        if s not in connected_more[p]:
                            connected_more[p].append(s)
               
                    else:
                        connected_more[p] = [s]

# for c in connected:
#     longest = ""
#     for item in connected[c]:
#         if len(item) > len(longest) and " BY " not in item and "REPLY" not in item and "ADDRESSES" not in item and "ARGUMENT" not in item and "REEXAMINATION" not in item:
#             if item.split()[-1] == c:
#                 longest = item
#     connected[c] = longest

In [7]:
print (len(connected), len(connected_more))

294 333


In [8]:
all_data = {'LastName': [], 'Variants': []}
for c in connected_more:
    all_data['LastName'].append(c)
    all_data['Variants'].append(connected_more[c])

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(all_data)

In [12]:
df

,LastName,Variants
0,SANDI,"[ADV SANDI, MR SANDI, MRS SANDI, MS SANDI, N S..."
1,MTHEMBU,"[MR MTHEMBU, ADV MTHEMBU, PETROS VUSUMUZI MTHE..."
2,TSEMANE,"[MR D TSEMANE, MR L TSEMANE, MR TSEMANE, MS TS..."
3,MOTATA,"[ADV MOTATA, MR MOTATA, JUDGE MOTATA]"
4,NDLOVU,"[ALFAS MABORE NDLOVU, MR NDLOVU, MRS NDLOVU, M..."
...,...,...
328,GOOSEN,"[MR GOOSEN, DR GOOSEN, ADRIAAN JACOBUS GOOSEN]"
329,KNOBEL,"[DANIEL PETER KNOBEL, COUNSEL FOR KNOBEL, DR K..."
330,NIENABER,"[MR COETZEE MR NIENABER, MR NIENABER, SAREL PE..."
331,CHILI,"[MD D CHILI, MR D CHILI, MR S CHILI, MRS B CHI..."


In [22]:
df.to_csv('../data/manual_validation_lastnames.csv', index=False)